In [26]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

In [16]:
df=pd.read_csv("../data_log_all.csv")

In [17]:
df.head()

,Usage_kWh,Lagging_Current_Reactive.Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2(tCO2),Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM,WeekStatus,Load_Type
0,0.068186,0.352315,0.0,0.0,0.638592,1.0,0.000000,0.0,0.0
1,0.070905,0.372114,0.0,0.0,0.585156,1.0,0.043478,0.0,0.0
2,0.072554,0.382602,0.0,0.0,0.557186,1.0,0.086957,0.0,0.0
3,0.068034,0.377244,0.0,0.0,0.562310,1.0,0.130435,0.0,0.0
4,0.077733,0.393285,0.0,0.0,0.537829,1.0,0.173913,0.0,0.0


In [18]:
# Sliding window transformation
def create_sliding_windows(data, window_size=1):
    X, y = [], []
    for i in range(len(data)-window_size):
        X.append(data[i:i+window_size, 1:])
        y.append(data[i+window_size, 0])
    return np.array(X), np.array(y)

In [19]:
data={}

In [20]:
df_np = df.to_numpy()

In [23]:
window_sizes=[1,4,8,12,16,24,36,48,74,168,336,672]

In [27]:
for window in window_sizes:
  X, y = create_sliding_windows(df_np, window)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.18, shuffle=False)
  data[f"win{window}"] = {
      "X_train": X_train,
      "X_test": X_test,
      "y_train": y_train,
      "y_test": y_test
  }

In [33]:
data['win1']['X_train'].shape, data['win1']['y_train'].shape, data['win1']['X_test'].shape, data['win1']['y_test'].shape    

((7182, 1, 8), (7182,), (1577, 1, 8), (1577,))

trainig

In [42]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Dropout

In [43]:
def build_lstm_model(model_type, input_shape):
    model = Sequential()

    if model_type == 'single':
        model.add(LSTM(64, input_shape=input_shape))
    elif model_type == 'double':
        model.add(LSTM(64, return_sequences=True, input_shape=input_shape))
        model.add(LSTM(64))
    elif model_type == 'bidirectional':
        model.add(Bidirectional(LSTM(64), input_shape=input_shape))

    model.add(Dropout(0.1))
    model.add(Dense(1))
    return model

In [45]:
import tensorflow.keras.backend as K
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_true - y_pred)))

Single layer


In [ ]:
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
import os

model_dir = '/content/drive/MyDrive/single_models'
os.makedirs(model_dir, exist_ok=True)

In [47]:
from tensorflow.keras.models import load_model

# Save model to disk
def save_model(model, window_size):
    path = f"{model_dir}/win{window_size}.h5"
    model.save(path)

# Load model from disk
def load_saved_model(window_size):
    path = f"{model_dir}/win{window_size}.h5"
    if os.path.exists(path):
        return load_model(path, custom_objects={'rmse': rmse})
    else:
        return None


In [ ]:
single_models = {}

for window_size in window_sizes:
    print(f"==> Window: {window_size}")

    X_train = data[f"win{window_size}"]["X_train"]
    X_test = data[f"win{window_size}"]["X_test"]
    y_train = data[f"win{window_size}"]["y_train"]
    y_test = data[f"win{window_size}"]["y_test"]

    print("-->", X_train.shape, y_train.shape, X_test.shape)

    # Try to load the model
    model = load_saved_model(window_size)

    if model is not None:
        print(f"✅ Loaded existing model for window {window_size}")
        history = None  # Since we didn't train now
    else:
        print(f"🚀 Training new model for window {window_size}")
        model = build_lstm_model('single', (window_size, X_train.shape[2]))
        model.compile(optimizer='adam', loss=rmse, metrics=['mae'])
        history = model.fit(
            X_train, y_train,
            epochs=50,
            batch_size=32,
            validation_split=0.2,
            verbose=1
        )
        save_model(model, window_size)
        print(f"💾 Model saved for window {window_size}")

    single_models[f"win{window_size}"] = {
        "model": model,
        "history": history
    }


In [ ]:
import os
from tensorflow.keras.models import load_model

single_layers_folder = "single layers"
model_files = [f for f in os.listdir(single_layers_folder) if f.endswith('.h5') or f.endswith('.keras')]

loaded_models = {}
for model_file in model_files:
    model_path = os.path.join(single_layers_folder, model_file)
    loaded_models[model_file] = load_model(model_path)